In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/processed-police-incidents/processed_police_incidents.csv


In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
data = pd.read_csv('/kaggle/input/processed-police-incidents/processed_police_incidents.csv')

/tmp/ipykernel_874/29424824.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/kaggle/input/processed-police-incidents/processed_police_incidents.csv')


In [4]:
print(data['Type  Location'].value_counts())

Type  Location
Highway, Street, Alley ETC            202164
Single Family Residence - Occupied    120919
Apartment Parking Lot                 111252
Apartment Complex/Building             83140
Parking (Business)                     77496
                                       ...  
School/College                           118
Farm Facility                             77
Playground                                58
Military Installation                     27
Tribal Lands                              11
Name: count, Length: 73, dtype: int64


In [5]:
data = data.dropna(subset=['X Coordinate', 'Y Cordinate', 'Council District', 'Type  Location'])

In [6]:
data

,Incident Number w/year,Year of Incident,Service Number ID,Watch,Call (911) Problem,Type of Incident,Type Location,Type of Property,Incident Address,Apartment Number,...,Update Date,X Coordinate,Y Cordinate,Zip Code,City,State,Location1,Incident_Score,Time Bin,Safety Score
0,203058-2022,2022,203058-2022-01,1,PSE/09 - THEFT,THEFT OF PROP (AUTO ACC) <$100 - (NOT EMP),Apartment Parking Lot,Motor Vehicle,7152 FAIR OAKS AVE,NaN,...,2022-11-10 11:15:06.0000000,2.503598e+06,7.005279e+06,75231.0,DALLAS,TX,"7152 FAIR OAKS AVE\nDALLAS, TX 75231\n(32.8730...",35.0,Night,Moderately Safe
1,264509-2016,2016,264509-2016-01,2,58 - ROUTINE INVESTIGATION,FALSE STATEMENT FOR PROPERTY/CREDIT $2500 < $30K,Single Family Residence - Occupied,NaN,8545 MIDPARK RD,NaN,...,2017-03-31 00:43:58.0000000,2.503120e+06,7.028109e+06,75240.0,DALLAS,TX,"8545 MIDPARK RD\nDALLAS, TX 75240\n(32.93652, ...",25.0,Afternoon,Safe
2,004110-2015,2015,004110-2015-01,2,58 - ROUTINE INVESTIGATION,CREDIT CARD OR DEBIT CARD ABUSE,Business Office,NaN,1400 S LAMAR ST,NaN,...,2016-12-16 10:29:24.0000000,2.492641e+06,6.966517e+06,75215.0,DALLAS,TX,"1400 S LAMAR ST\nDALLAS, TX 75215\n(32.76737, ...",65.0,Afternoon,Moderately Unsafe
3,022647-2023,2023,022647-2023-01,1,58 - ROUTINE INVESTIGATION,ABANDONED PROPERTY (NO OFFENSE),Outdoor Area Public/Private,NaN,5509 DRANE DR,NaN,...,2023-02-13 13:23:53.0000000,2.485560e+06,6.993803e+06,75209.0,DALLAS,TX,"5509 DRANE DR\nDALLAS, TX 75209\n(32.84277, -9...",10.0,Early Morning,Safe
4,112585-2015,2015,112585-2015-01,3,09V-01 UUMV JUST OCRD,UNAUTHORIZED USE OF MOTOR VEH - AUTOMOBILE,Parking Lot (All Others),NaN,906 BISON TRL,NaN,...,2019-02-18 18:50:53.0000000,2.474558e+06,6.960968e+06,75208.0,DALLAS,TX,"906 BISON TRL\nDALLAS, TX 75208\n(32.75337, -9...",65.0,Afternoon,Moderately Unsafe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1122599,276915-2016,2016,276915-2016-01,2,DASF-DIST ACTIVE SHOOTER FOOT,"THEFT OF PROP > OR EQUAL $750 BUT <$2,500 -SHO...",Convenience Store,NaN,4855 N CENTRAL EXPY,NaN,...,2017-08-21 09:23:26.0000000,2.495400e+06,6.987855e+06,75205.0,DALLAS,TX,"4855 N CENTRAL EXPY\nDALLAS, TX 75205\n(32.826...",65.0,Morning,Moderately Unsafe
1122600,163009-2017,2017,163009-2017-01,1,11B - BURG OF BUS,BURGLARY OF BUILDING - FORCED ENTRY,Storage Facility,Parking Lot,1919 N HASKELL AVE,NaN,...,2017-07-25 11:16:45.0000000,2.494964e+06,6.978938e+06,75204.0,DALLAS,TX,"1919 N HASKELL AVE\nDALLAS, TX 75204\n(32.8019...",85.0,Night,Very Unsafe
1122601,265586-2015,2015,265586-2015-01,3,55 - TRAFFIC STOP,POSS MARIJUANA <2OZ,Parking Lot (All Others),NaN,8687 N CENTRAL SERV W,NaN,...,2018-02-19 09:22:43.0000000,2.499426e+06,7.002954e+06,75225.0,DALLAS,TX,"8687 N CENTRAL SERV W\nDALLAS, TX 75225\n(32.8...",30.0,Afternoon,Moderately Safe
1122602,192459-2015,2015,192459-2015-01,1,31 - CRIMINAL MISCHIEF,"CRIM MISCHIEF > OR EQUAL $500 BUT < $1,500",Apartment Parking Lot,NaN,7325 DULANEY DR,NaN,...,2015-08-25 07:58:14.0000000,2.514860e+06,6.977693e+06,75228.0,DALLAS,TX,"7325 DULANEY DR\nDALLAS, TX 75228\n(32.7974662...",55.0,Night,Moderately Safe


In [7]:
# Perform label encoding for categorical columns (Day of the Week and Time Bin)
label_encoder = LabelEncoder()
data['Day1 of the Week'] = label_encoder.fit_transform(data['Day1 of the Week'])
data['Time Bin'] = label_encoder.fit_transform(data['Time Bin'])
data['Division'] = label_encoder.fit_transform(data['Division'])
data['Safety Score'] = label_encoder.fit_transform(data['Safety Score'])
data['Council District'] = label_encoder.fit_transform(data['Council District'])
data['Type  Location'] = label_encoder.fit_transform(data['Type  Location'])
data['Month1 of Occurence'] = label_encoder.fit_transform(data['Month1 of Occurence'])

In [8]:
# Define the features and target variable
features = ['Day1 of the Week', 'Time Bin' , 'Zip Code', 'Sector', 'Division', 'X Coordinate', 'Y Cordinate', 'Council District', 'Type  Location', 'Month1 of Occurence']
target = 'Safety Score'

In [9]:
# # Split the dataset into training and testing sets
# X = data[features]
# y = data[target]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [10]:
# Assuming you have a DataFrame 'data' with columns 'features' and 'target'
# Group the data by 'zipcode' and split each group into train and test
train_set = pd.DataFrame()
test_set = pd.DataFrame()

splits = {}

unique_zipcodes = data['Zip Code'].unique()

for zipcode in unique_zipcodes:
    group = data[data['Zip Code'] == zipcode]
    n_rows = len(group)
    
    # Check if there are less than 2 entries for the zipcode
    if n_rows < 2:
        splits[zipcode] = [group, pd.DataFrame(columns=data.columns)]
    else:
        X_train, X_test = train_test_split(group, test_size=0.2, random_state=42)
        splits[zipcode] = [X_train, X_test]

# Combine the splits for all unique Zip Codes
train_set = pd.concat([split[0] for split in splits.values()])
test_set = pd.concat([split[1] for split in splits.values()])


In [11]:
# Separate your features and target columns
X_train = train_set[features]
y_train = train_set[target]
X_test = test_set[features]
y_test = test_set[target]

In [12]:
X_train

,Day1 of the Week,Time Bin,Zip Code,Sector,Division,X Coordinate,Y Cordinate,Council District,Type Location,Month1 of Occurence
772519,6,3,75231.0,210.0,2,2.499868e+06,7.009987e+06,8,50,9
252281,1,2,75231.0,210.0,2,2.504097e+06,7.005453e+06,8,5,5
821302,5,0,75231.0,210.0,2,2.500295e+06,7.009310e+06,8,49,4
87653,1,1,75231.0,210.0,2,2.502498e+06,7.004062e+06,8,5,6
579507,4,3,75231.0,210.0,2,2.499668e+06,7.004971e+06,8,60,11
...,...,...,...,...,...,...,...,...,...,...
1081538,2,0,78501.0,640.0,1,2.501957e+06,7.026851e+06,6,34,7
1094726,0,0,50704.0,640.0,1,2.501957e+06,7.026851e+06,6,31,6
1097890,0,2,57238.0,210.0,2,2.503166e+06,7.001334e+06,17,37,1
1117026,1,1,72231.0,240.0,2,2.510242e+06,7.010465e+06,5,37,1


In [13]:
y_test

366177     0
1099761    1
1041147    0
600839     1
684139     0
          ..
718092     2
708392     1
647014     1
893509     1
916017     1
Name: Safety Score, Length: 224194, dtype: object

In [14]:
y_pred

NameError: name 'y_pred' is not defined

In [15]:
# Create and train a Random Forest Classifier model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

import pickle
filename = 'rfc_model.pkl'
pickle.dump(model, open(filename, 'wb'))

# Make predictions on the test set
y_pred = model.predict(X_test)
y_test = pd.to_numeric(y_test, errors='coerce')
# Evaluate the model using accuracy score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.522261969544234


In [ ]:
# from sklearn.ensemble import GradientBoostingClassifier
# # Initialize the Gradient Boosting Classifier
# gb_classifier = GradientBoostingClassifier(n_estimators=100, random_state=42)  # You can adjust the hyperparameters as needed
# gb_classifier.fit(X_train, y_train)

# import pickle
# filename = 'rfc_model.pkl'
# pickle.dump(gb_classifier, open(filename, 'wb'))

# # Make predictions on the test set
# y_pred = gb_classifier.predict(X_test)

# # Evaluate the model using accuracy score
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy: {accuracy}")

In [ ]:
# # Assuming you have a new data point in the form of a dictionary
# new_data = {
#     'Day1 of the Week': 'Tue',   # Example value
#     'Time Integer': 'Afternoon',        # Example value
# }


# # Convert the new data into a DataFrame with the same structure as your training data
# new_data_df = pd.DataFrame([new_data])

# # Perform label encoding for categorical columns (Day of the Week and Time Bin)
# new_data_df['Day1 of the Week'] = 1
# new_data_df['Time Integer'] = 20

# # Perform one-hot encoding for the Zip Code column
# # new_data_df = pd.get_dummies(new_data_df, columns=['Zip Code'], prefix='Zip')



In [ ]:
# new_data_df

In [ ]:
# # Initialize a dictionary to hold the one-hot encoded values
# encoded_values = {}
# unique_zip_codes.sort()
# # Create one-hot encoded columns for all unique Zip Codes seen in training data
# for col in unique_zip_codes:
#     encoded_values['Zip_' + str(col)] = [True if col == '75231.0' else False]


In [ ]:
# print(encoded_values)

In [ ]:
# encoded_df = pd.DataFrame(encoded_values)
# new_data_df = pd.concat([new_data_df, encoded_df], axis=1)

In [ ]:
# encoded_df

In [ ]:
# new_data_df

In [ ]:
# # Use your trained model to predict the 'Safety Score' for the new data
# predicted_safety_score = model.predict([[75231, 1, 1]])

# # Print the predicted 'Safety Score'
# print(f"Predicted Safety Score: {predicted_safety_score[0]}")